# Set up environment

In [ ]:
import sys
print(sys.prefix)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import urllib, os,sys, pdfplumber, glob, requests, wordcloud, re, dateparser

# Set up working dir

In [ ]:
base_dir = os.path.realpath('../..')
print(base_dir)
data_dir = base_dir + '\Data'

In [ ]:
in_dir = data_dir + '\\TK_commissieVWS\\auto_download_20230118'

## Load data

In [ ]:
vws_data = pd.read_csv(in_dir + '\\speaking_turns_coded.csv', index_col = 0)

In [ ]:
vws_data.columns

## Load lexicon

In [ ]:
lex_dict = {'original':pd.read_csv(data_dir + '\\lexicons\\lexicon.csv',  index_col = 0),
            'sens_noSEGV':pd.read_csv(data_dir + '\\lexicons\\lexicon_no_SEGV.csv', index_col = 0)}

## Find terms in data

##### apply fun to speaking turns and identify categories

In [ ]:
for lex_name,lexicon_use in lex_dict.items():
    
    all_speaking_turns_labeled = vws_data.copy()
    
    print(lex_name)
    print('Creating category count placeholders')
    all_speaking_turns_labeled['lexicon'] = lex_name
    all_speaking_turns_labeled['LEEF'] = np.zeros([all_speaking_turns_labeled.shape[0],1])
    all_speaking_turns_labeled['SDOH'] = np.zeros([all_speaking_turns_labeled.shape[0],1])

    for ci,cat in enumerate(lexicon_use['category'].unique()):

        print('Category: %s.'%cat)
        cat_lexicon = lexicon_use.loc[lexicon_use['category']==cat,:].copy().reset_index(drop=True)

        for ti,term_info in cat_lexicon.iterrows():

            # Load term
            term = term_info['term']
            term_regex =  term_info['regex_search']
            print('Term: %s. Pattern: %s.'%(term, term_regex))

            turns_with_term_i = (all_speaking_turns_labeled['text'].str.count(term_regex).values > 0).astype(int)
            print('%i turns with term found'%sum(turns_with_term_i))

            all_speaking_turns_labeled[term] = turns_with_term_i
            all_speaking_turns_labeled[cat] = np.maximum(all_speaking_turns_labeled[cat],turns_with_term_i).astype(int)
    
    # Save
    print('total counts:')
    print(all_speaking_turns_labeled[['SDOH','LEEF']].sum(axis=0))
    print('Saving table with shape %s, %s\n\n'%all_speaking_turns_labeled.shape)
    all_speaking_turns_labeled.to_csv(in_dir + 
              '\\speaking_turns_coded_labeled-%s.csv'%lex_name)